In [27]:
import json
import pandas as pd 
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [28]:
# load 
with open('api_定义.json') as f:
    api_dict = json.load(f)
df_api = pd.DataFrame(api_dict)

In [29]:
# preprocess

def get_real_param_num(parameters:dict):
    num = len(parameters['required'])
    if '运算符' in parameters['properties']:
        if '固定为' in parameters['properties']['运算符']['description']:   # 有运算符，但是固定为某个值
            num -= 1
    return num
df_api['param_names'] = df_api['parameters'].apply(lambda d:d['required'])
df_api['param_num'] = df_api['parameters'].apply(get_real_param_num)
df_api.drop(columns=['parameters'], inplace=True)
df_api.api_description = df_api.api_description.str.replace('这个API接口的功能是', '').replace('这个API的功能是', '')

In [30]:
df_api.head()

,category_name,tool_name,api_name,api_description,param_names,param_num
0,基金,条件选基,查询代码,接收运算符和基金代码作为输入，返回与输入的基金代码匹配的基金详细信息。,"[运算符, 基金代码]",1
1,基金,条件选基,查询类型,接收运算符和基金类型作为输入，返回相应类型的基金列表。通过这个接口，用户可以查询到该类型的所有基金。,"[运算符, 基金类型]",1
2,基金,条件选基,查询规模,接收运算符和基金规模作为输入，并返回与给定条件相符的基金名称列表。,"[运算符, 规模]",1
3,基金,条件选基,查询晨星评级,接收运算符和晨星评级作为输入，返回对应晨星评级的基金列表。,"[运算符, 晨星评级]",1
4,基金,条件选基,查询蚂蚁评级,接收运算符和蚂蚁评级作为输入，返回具有指定评级的基金列表。,"[运算符, 评级]",1


## 1. 分类概览

In [31]:
pd.DataFrame(df_api.groupby(['category_name']).tool_name.value_counts())

tool_name
category_name tool_name           
基金            基金查询              76
              条件选基              76
股票            股票查询              50
              条件选股              49
通用            数值计算               4
              逻辑计算               2

**注意：** 同一个 api_name 可能在不同tool_name中出现，本质上是两个不同的 api
如
```json
    {
        "category_name": "基金",
        "tool_name": "条件选基",
        "api_name": "查询申购费率",
        "api_description": "这个API接口的功能是接收运算符和申购费率作为输入，返回满足条件的基金列表。用户可以通过这个接口查询符合特定申购费率条件的基金。",
        "parameters": {
            "properties": {
                "运算符": {
                    "type": "string",
                    "description": "运算符号，候选值是大于、小于、等于"
                },
                "申购费率": {
                    "type": "float",
                    "description": "基金的申购费率"
                }
            },
            "required": [
                "运算符",
                "申购费率"
            ]
        }
    },
    {
        "category_name": "基金",
        "tool_name": "基金查询",
        "api_name": "查询申购费率",
        "api_description": "这个API接口的功能是接收一组基金代码列表作为输入，返回每个基金代码对应的申购费率。用户可以通过这个接口快速查询到具体基金的申购费率，以及进行进一步的基金分析和操作。",
        "parameters": {
            "properties": {
                "基金代码列表": {
                    "type": "list",
                    "description": "用户需要提供一个包含一个或多个基金代码的数组，每个基金代码指定了用户想查询的具体基金。这个列表是API的输入数据，每个基金代码应该是一个有效且可识别，确保API能正确返回所需数据。"
                }
            },
            "required": [
                "基金代码列表"
            ]
        }
    }
```

## 2. 各类详情

### 2.1 基金查询

- 问某个具体的基金名字 一般需要通过 `查询代码`(76) api 查到相应的代码， 在进行后续操作
- 其他查询api都是 接受 `基金代码` | `基金经理` 作为核心入参，偶尔辅以`时间`

In [32]:
fund_inquiry_df = df_api[df_api.tool_name=='基金查询']
fund_inquiry_df

,category_name,tool_name,api_name,api_description,param_names,param_num
76,基金,基金查询,查询代码,接收标准基金名称作为输入，返回对应的具体基金代码。用户可以通过这个接口根据基金名称快速查询到具体的基金代码，然后进行后续的基金信息查询及操作。,[基金名称],1
77,基金,基金查询,查询类型,接收一组基金代码列表作为输入，返回每个基金代码对应的基金类型，比如“混合型”、“货币型”等。用户可以通过这个接口快速查询到具体基金的类型，从而进行进一步的基金分析和操作。,[基金代码列表],1
78,基金,基金查询,查询规模,接收一组基金代码列表作为输入，返回每个基金代码对应的基金规模，用户可以通过这个接口快速查询到具体基金的规模大小，从而进行进一步的基金分析和操作。,[基金代码列表],1
79,基金,基金查询,查询晨星评级,接收一组基金代码列表作为输入，返回每个基金代码对应的晨星评级。晨星评级是一种评估基金综合实力的评级系统，对基金的业绩和风险进行综合评比，用星级表示，五星为最高，一星为最低。,[基金代码列表],1
80,基金,基金查询,查询蚂蚁评级,接收一组基金代码列表作为输入，返回每个基金代码对应的蚂蚁评级。蚂蚁评级是一种评价基金业绩和风险的评估指标，用户可以通过这个接口了解到每个基金的业绩表现和风险水平。,[基金代码列表],1
81,基金,基金查询,查询申购费率,接收一组基金代码列表作为输入，返回每个基金代码对应的申购费率。用户可以通过这个接口快速查询到具体基金的申购费率，以及进行进一步的基金分析和操作。,[基金代码列表],1
82,基金,基金查询,查询成立年限,接收一组基金代码列表作为输入，返回每个基金代码对应的成立年限。这个接口可以帮助用户了解基金的历史表现和稳定性，同时也可以作为评估基金管理人能力的一个参考。,[基金代码列表],1
83,基金,基金查询,查询开放类型,接收一组基金代码列表作为输入，返回每个基金代码对应的开放类型，例如“开放型”、“封闭型”等。用户可以通过这个接口快速查询到具体基金的开放类型，以便进行进一步的基金选择和配置。,[基金代码列表],1
84,基金,基金查询,查询申购状态,接收基金代码列表作为输入，返回每个基金代码对应的基金申购状态，如“开放申购”、“暂停申购”等。通过这个接口，用户可以快速查询具体基金的申购状态，以进行投资决策。,[基金代码列表],1
85,基金,基金查询,查询蚂蚁金选标识,接收一组基金代码列表作为输入，返回每个基金代码是否被蚂蚁金服选中作为优选基金，从而帮助用户了解该基金在蚂蚁金服平台的评价和认可度。,[基金代码列表],1


### 2.2 条件选基

- 多数入参形式为：`运算符` + `某个指标` 构成， 返回满足条件的 **基金名称列表**  **基金列表**？ **基金经理列表**等
- 这些api名称都是 `查询xxx` 的形式，有些歧义，是否需要在这里做文章

In [33]:
fund_select_df = df_api[df_api.tool_name=='条件选基']
fund_select_df

,category_name,tool_name,api_name,api_description,param_names,param_num
0,基金,条件选基,查询代码,接收运算符和基金代码作为输入，返回与输入的基金代码匹配的基金详细信息。,"[运算符, 基金代码]",1
1,基金,条件选基,查询类型,接收运算符和基金类型作为输入，返回相应类型的基金列表。通过这个接口，用户可以查询到该类型的所有基金。,"[运算符, 基金类型]",1
2,基金,条件选基,查询规模,接收运算符和基金规模作为输入，并返回与给定条件相符的基金名称列表。,"[运算符, 规模]",1
3,基金,条件选基,查询晨星评级,接收运算符和晨星评级作为输入，返回对应晨星评级的基金列表。,"[运算符, 晨星评级]",1
4,基金,条件选基,查询蚂蚁评级,接收运算符和蚂蚁评级作为输入，返回具有指定评级的基金列表。,"[运算符, 评级]",1
5,基金,条件选基,查询申购费率,接收运算符和申购费率作为输入，返回满足条件的基金列表。用户可以通过这个接口查询符合特定申购费率条件的基金。,"[运算符, 申购费率]",2
6,基金,条件选基,查询成立年限,接收运算符和基金的成立日期作为输入，返回满足条件的基金名称列表。,"[运算符, 成立日期]",2
7,基金,条件选基,查询开放类型,接收运算符和基金的开放类型作为输入，返回属于该开放类型的基金名称列表。,"[运算符, 开放类型]",1
8,基金,条件选基,查询申购状态,根据申购状态和运算符来筛选基金，返回符合该状态的基金列表。用户可以通过这个接口查询具有特定申购状态的基金。,"[运算符, 申购状态]",1
9,基金,条件选基,查询蚂蚁金选标识,接收运算符和蚂蚁金选标识作为输入，返回符合该标识条件的基金名称列表。蚂蚁金选标识表示该基金是否被蚂蚁金服平台选中为推荐产品。,"[运算符, 蚂蚁金选标识]",1


### 2.3 股票查询

- 与基金查询类似，`201-股票查询` 根据名称查代码，其余均为根据代码查信息

In [33]:
stock_query_df = df_api[df_api.tool_name=='股票查询']
stock_query_df

,category_name,tool_name,api_name,api_description,param_names,param_num
201,股票,股票查询,查询代码,这个API接口的功能是接收一组标准股票名称作为输入，返回每个股票名称对应的股票代码。这个API有助于用户通过股票名称找到对应的股票代码，从而进行进一步的股票分析和操作。,[股票名称列表],1
202,股票,股票查询,查询开盘价,这个API接口的功能是接收股票代码列表和指定时间作为输入，返回该股票在指定时间的开盘价。,"[股票代码列表, 时间]",2
203,股票,股票查询,查询最高价,这个API接口的功能是接收股票代码列表和指定时间作为输入，返回该股票某个时间的最高价,"[股票代码列表, 时间]",2
204,股票,股票查询,查询最低价,这个API接口的功能是接收股票代码列表和指定时间作为输入，返回该股票某个时间的最低价,"[股票代码列表, 时间]",2
205,股票,股票查询,查询当前价,这个API接口的功能是接收股票代码列表作为输入，返回该股票的当前价格。,[股票代码列表],1
206,股票,股票查询,查询收盘价,这个API接口的功能是接收股票代码列表和指定时间作为输入，返回该股票某个时间的收盘价,"[股票代码列表, 时间]",2
207,股票,股票查询,查询成交量,这个API接口的功能是接收股票代码列表和指定时间作为输入，返回每个股票在指定时间的成交量。用户可以通过这个接口快速查询到具体股票的成交量，从而进行进一步的股票分析和操作。,"[股票代码列表, 时间]",2
208,股票,股票查询,查询成交额,这个API接口的功能是接收股票代码列表和指定时间作为输入参数，返回某一指定时间的股票的成交额。,"[股票代码列表, 时间]",2
209,股票,股票查询,查询涨停价,这个API接口的功能是接收股票代码列表和指定时间作为输入，返回该股票当天的涨停价,"[股票代码列表, 时间]",2
210,股票,股票查询,查询跌停价,这个API接口的功能是接收股票代码列表和指定时间作为输入，返回该股票某个时间的跌停价,"[股票代码列表, 时间]",2


### 2.4 条件选股

In [34]:
stock_select_df = df_api[df_api.tool_name=='条件选股']
stock_select_df

,category_name,tool_name,api_name,api_description,param_names,param_num
152,股票,条件选股,查询代码,这个API接口的功能是接收运算符和股票代码作为输入，返回符合该代码条件的股票名称。用户可以通过这个接口查询特定股票代码所对应的股票。,"[运算符, 股票代码]",1
153,股票,条件选股,查询开盘价,这个API接口的功能是接收运算符、一定的开盘价、时间作为输入，返回在指定时间内满足开盘价条件的股票名称列表。,"[运算符, 开盘价, 时间]",3
154,股票,条件选股,查询最高价,这个API接口的功能是接收运算符、最高价、时间作为输入，返回在指定时间段内满足最高价条件的股票名称列表。,"[运算符, 最高价, 时间]",3
155,股票,条件选股,查询最低价,这个API接口的功能是接收运算符、最低价、时间作为输入，返回在指定时间内满足最低价条件的股票名称列表。,"[运算符, 最低价, 时间]",3
156,股票,条件选股,查询当前价,这个API接口的功能是接收运算符和当前价格作为输入，返回符合这个价格条件的股票列表。,"[运算符, 当前价]",2
157,股票,条件选股,查询收盘价,这个API接口的功能是接收运算符、某个具体的收盘价数值、时间为输入条件，返回在指定时间内收盘价符合该条件的股票名称列表。,"[运算符, 收盘价, 时间]",3
158,股票,条件选股,查询成交量,这个API接口的功能是接收运算符、成交量、时间作为输入，返回在指定时间内满足成交量条件的股票名称列表。,"[运算符, 成交量, 时间]",3
159,股票,条件选股,查询成交额,这个API接口的功能是接收运算符、成交额、时间作为输入，返回在指定时间内满足成交额条件的股票名称列表。,"[运算符, 成交额, 时间]",3
160,股票,条件选股,查询涨停价,这个API接口的功能是接收运算符、涨停价、时间作为输入，返回在特定交易日涨停价符合条件的股票列表。,"[运算符, 涨停价, 时间]",3
161,股票,条件选股,查询跌停价,这个API接口的功能是接收运算符、跌停价和时间（今日）作为输入，返回在当日触及跌停价格条件的股票名称列表。,"[运算符, 跌停价, 时间]",3


### 2.5 通用

In [35]:
stock_select_df = df_api[df_api.category_name=='通用']
stock_select_df

,category_name,tool_name,api_name,api_description,param_names,param_num
251,通用,数值计算,加法计算,这个API接口的功能是接收两个数作为输入，返回两数之和。,"[被加数, 加数]",2
252,通用,数值计算,减法计算,这个API接口的功能是接收两个数作为输入，返回两数之差。,"[被减数, 减数]",2
253,通用,数值计算,乘法计算,这个API接口的功能是接收两个数作为输入，返回两数之积。,"[被乘数, 乘数]",2
254,通用,数值计算,除法计算,这个API接口的功能是接收两个数作为输入，返回被除数/除数的结果。,"[被除数, 除数]",2
255,通用,逻辑计算,与计算,这个API接口的功能是接收两个列表作为输入，返回取交集的列表。,"[列表1, 列表2]",2
256,通用,逻辑计算,或计算,这个API接口的功能是接收两个列表作为输入，返回取并集的列表。,"[列表1, 列表2]",2
